In [14]:
from surprise import Dataset
import numpy as np
from surprise.model_selection.split import KFold
from sklearn.metrics import mean_absolute_error as mae
import time
import statistics as stats

# Implementation

In [2]:
start = None
end = None
def tick():
    global start
    start = time.time()

def tock(prefix=""):
    global start, end
    end = time.time()
    print(prefix,"TIME:", end - start)

In [20]:
n_rating_values = 5

def get_bit_rating_matrix(all_ratings, n_items, n_users):
    bit_rating_matrix = np.zeros((n_items * n_rating_values, n_users), dtype=bool)
    for (u, i, rating) in all_ratings:
        index = int(i * n_rating_values + rating - 1)
        bit_rating_matrix[index, u] = 1
    return np.array(bit_rating_matrix, dtype=bool)

def get_support(bit_rating_matrix):
    return np.count_nonzero(bit_rating_matrix, axis = 1)

def mining_frequent_itemset(bit_rating_matrix, min_support = 1, improved = True):
    support = get_support(bit_rating_matrix)
    asc_order = np.argsort(support)
    desc_order = np.flip(asc_order)
    n_rows = len(bit_rating_matrix)
    bit_rating_matrix = np.append(np.arange(n_rows).reshape(n_rows,1), bit_rating_matrix, axis=1)
    O = bit_rating_matrix[desc_order]
    n_items_above_treshold = np.count_nonzero(support >= min_support)
    O = O[:n_items_above_treshold]

    S = []
    while True:
        if len(O) <= 1:
            break
        j = 0
        c = O[j, :]
        si = [c[0]]
        si_pattern = c[1:]
        O = np.delete(O, j, 0)
        while True:
            if j == len(O):
                S.append(si)
                O = np.array([x for x in O.tolist() if x[0] not in si])
                print(len(O))
                break
            else:
                if improved:
                    best_j = j
                    best_support = -1
                    # for k in range(j, min(j+1, len(O))):
                    for k in range(j, len(O)):
                        c = O[k, :]
                        new_si_pattern = np.logical_and(si_pattern, c[1:])
                        new_support = np.count_nonzero(new_si_pattern)
                        if new_support > best_support:
                            best_j = k
                            best_support = new_support
                            if new_support == np.count_nonzero(si_pattern):
                                break
                        
                    c = O[best_j, :]
                    new_si_pattern = np.logical_and(si_pattern, c[1:])
                    if np.count_nonzero(new_si_pattern) >= min_support:
                        O[best_j, :], O[j,:] = O[j,:], O[best_j, :]
                        si_pattern = new_si_pattern
                        si.append(c[0])
                else:
                    c = O[j, :]
                    new_si_pattern = np.logical_and(si_pattern, c[1:])
                    if np.count_nonzero(new_si_pattern) >= min_support:
                        si_pattern = new_si_pattern
                        si.append(c[0])
            j += 1
    return S

def bitset(s, n_rows):
    new_s = np.zeros(n_rows, dtype=bool)
    new_s[s] = True
    return new_s

def recommend(B, bit_users):
    tick()
    S = mining_frequent_itemset(B)
    tock("MINING")
    n_rows = B.shape[0]
    
    tick()
    result = []
    for i in range(bit_users.shape[1]):
        bit_u = bit_users[:, i]
        bit_matched_itemset = None
        max_count = -1
        for s in S:
            bit_s = bitset(s, n_rows)
            bs = np.logical_and(bit_u, bit_s)
            # if np.array_equal(bs,bit_u) and np.count_nonzero(bs) > max_count:
            if np.count_nonzero(bs) > max_count:
                bit_matched_itemset = bit_s
                max_count = np.count_nonzero(bs)
        r_item = np.logical_and(bit_matched_itemset, np.logical_not(bit_u))
        result.append(r_item)
    tock("PREDICTION")
    return np.array(result).T

def get_folds(n_splits=5, random_state=None):
    kf = KFold(n_splits=n_splits, random_state=random_state)
    folds = []
    for fold in kf.split(data):
        test_data = fold[1]
        for i in range(len(test_data)):
            user, item, rating = test_data[i]
            fold[1][i] = tuple([int(user)-1, int(item)-1, int(rating)])
        folds.append(fold)
    return folds

def bit_to_normal(bit_matrix):
    rating_matrix = []
    for c in range(bit_matrix.shape[1]):
        ratings = []
        rating = 0 if bit_matrix[0, c] == False else 1
        for r in range(1, bit_matrix.shape[0]):
            if bit_matrix[r,c]:
                rating = r % 5 + 1
            if (r+1) % 5 == 0:
                ratings.append(rating)
                rating = 0
        rating_matrix.append(ratings)
    return np.array(rating_matrix)

# Validation

In [23]:
data = Dataset.load_builtin('ml-100k')
folds = get_folds(random_state=19)

mae_array = []

for fold in folds:
    trainset = fold[0]
    test_ratings = fold[1]
    n_items = data.build_full_trainset().n_items
    n_users = data.build_full_trainset().n_users
    bit_train = get_bit_rating_matrix(trainset.all_ratings(), n_items, n_users)
    bit_test = get_bit_rating_matrix(test_ratings, n_items, n_users)

    bit_result = recommend(bit_train, bit_test)

    result = bit_to_normal(bit_result)
    test_ratings = bit_to_normal(bit_test)  
    
    pred_ratings = result[test_ratings > 0]
    real_ratings = test_ratings[test_ratings > 0]

    real_ratings = real_ratings[pred_ratings>0]
    pred_ratings = pred_ratings[pred_ratings>0]

    error = mae(real_ratings, pred_ratings)
    print("MAE: ", error)
    mae_array.append(error)
mae_array

In [17]:
pred_time_array = [7.512872219085693, 7.73358416557312, 7.610635042190552, 7.3360631465911865, 7.929634094238281]
min_time_array = [27.227344751358032, 27.273317337036133, 29.421992778778076, 25.278998136520386, 28.51817297935486]
n_users = data.build_full_trainset().n_users
print("Mean prediction time: {}+-{}".format(stats.mean(pred_time_array)/n_users, stats.stdev(pred_time_array)/n_users))
print("Mean mining time: {}+-{}".format(stats.mean(min_time_array), stats.stdev(min_time_array)))
print("MAE: {}+-{}".format(stats.mean(mae_array), stats.stdev(mae_array)))

Mean prediction time: 0.00808542707692022+-0.0002376290894201747
Mean mining time: 27.5439651966095+-1.563621034400534
MAE: 1.6071724090271768+-0.04009661608595808
